## ⾃动求梯度

In [1]:
from mxnet import autograd, nd

### 简单例⼦
对函数 $y = 2x^⊤x$ 求关于列向量 $x$ 的梯度。
1. 我们先创建变量x，并赋初值
2. 为了求有关变量x的梯度，我们需要先调⽤`attach_grad`函数来申请存储梯度所需要的内存
3. 下⾯定义有关变量x的函数。为了减少计算和内存开销，默认条件下MXNet不会记录⽤于求梯度
的计算。我们需要调⽤record函数来要求MXNet记录与求梯度有关的计算
4. 由于x的形状为（4, 1），y是⼀个标量。接下来我们可以通过调⽤backward函数⾃动求梯度。需
要注意的是，如果y不是⼀个标量，MXNet将默认先对y中元素求和得到新的变量，再求该变量有
关x的梯度

In [2]:
x = nd.arange(4).reshape(4,1)
x


[[0.]
 [1.]
 [2.]
 [3.]]
<NDArray 4x1 @cpu(0)>

In [3]:
x.attach_grad()

In [4]:
with(autograd.record()):
    y = 2*nd.dot(x.T, x)

In [5]:
y.backward()    

函数 $y = 2x^⊤x$ 关于$x$ 的梯度应为$4x$。现在我们来验证⼀下求出来的梯度是正确的

In [8]:
assert (x.grad - 4*x).norm().asscalar() == 0
x.grad


[[ 0.]
 [ 4.]
 [ 8.]
 [12.]]
<NDArray 4x1 @cpu(0)>

### 训练模式和预测模式
在调⽤record函数后，MXNet会记录并计算梯度。此外，默认情况下autograd还
会将运⾏模式从预测模式转为训练模式。这可以通过调⽤is_training函数来查看

In [10]:
print(autograd.is_training())
with(autograd.record()):
    print(autograd.is_training())

False
True


### 对Python控制流求梯度
使⽤MXNet的⼀个便利之处是，即使函数的计算图包含了Python的控制流（如条件和循环控制），
我们也有可能对变量求梯度。
考虑下⾯程序，其中包含Python的条件和循环控制。需要强调的是，这⾥循环（while循环）迭
代的次数和条件判断（if语句）的执⾏都取决于输⼊a的值。

In [11]:
def f(a):
    b = a * 2
    while b.norm().asscalar() < 1000:
        b = b * 2
        if b.sum().asscalar() > 0:
            c = b
        else:
            c = 100 * b
    return c

我们像之前⼀样使⽤record函数记录计算，并调⽤backward函数求梯度。

In [12]:
a = nd.random.normal(shape=1)
a.attach_grad()
with(autograd.record()):
    c = f(a)
c.backward()    

In [14]:
a.grad == c/a


[1.]
<NDArray 1 @cpu(0)>

- MXNet提供`autograd`模块来⾃动化求导过程。
- MXNet的`autograd`模块可以对⼀般的命令式程序进⾏求导。
- MXNet的运⾏模式包括训练模式和预测模式。我可以通过`autograd.is_training()`来判断运⾏模式

## 查阅文档
### 查找模块⾥的所有函数和类
当我们想知道⼀个模块⾥⾯提供了哪些可以调⽤的函数和类的时候，可以使⽤`dir`函数。下⾯我
们打印`nd.random`模块中所有的成员或属性

In [15]:
print(dir(nd.random))

['NDArray', '_Null', '__all__', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__spec__', '_internal', '_random_helper', 'current_context', 'exponential', 'exponential_like', 'gamma', 'gamma_like', 'generalized_negative_binomial', 'generalized_negative_binomial_like', 'multinomial', 'negative_binomial', 'negative_binomial_like', 'normal', 'normal_like', 'numeric_types', 'poisson', 'poisson_like', 'randint', 'randn', 'shuffle', 'uniform', 'uniform_like']


### 查找特定函数和类的使⽤
想了解某个函数或者类的具体⽤法时，可以使⽤`help`函数。让我们以NDArray中的`ones_like`函
数为例，查阅它的⽤法

In [16]:
help(nd.ones_like)

Help on function ones_like:

ones_like(data=None, out=None, name=None, **kwargs)
    Return an array of ones with the same shape and type
    as the input array.
    
    Examples::
    
      x = [[ 0.,  0.,  0.],
           [ 0.,  0.,  0.]]
    
      ones_like(x) = [[ 1.,  1.,  1.],
                      [ 1.,  1.,  1.]]
    
    
    
    Parameters
    ----------
    data : NDArray
        The input
    
    out : NDArray, optional
        The output NDArray to hold the result.
    
    Returns
    -------
    out : NDArray or list of NDArrays
        The output of this function.



In [17]:
x = nd.array([[0, 0, 0], [2, 2, 2]])
y = x.ones_like()
y


[[1. 1. 1.]
 [1. 1. 1.]]
<NDArray 2x3 @cpu(0)>

在Jupyter记事本⾥，我们可以使⽤?来将⽂档显⽰在另外⼀个窗口中。例如，使⽤nd.random.
uniform?将得到与help(nd.random.uniform)⼏乎⼀样的内容，但会显⽰在额外窗口⾥。此
外，如果使⽤nd.random.uniform??，那么会额外显⽰该函数实现的代码

In [21]:
nd.uniform?

In [22]:
nd.uniform??

### 在MXNet⽹站上查阅
可以在MXNet的⽹站上查阅相关⽂档。访问[MXNet⽹站](http://mxnet.apache.org/)
点击⽹⻚顶部的下拉菜单“API”可查阅各个前端语⾔的接口。此外，也可以在⽹⻚
右上⽅含“Search”字样的搜索框中直接搜索函数或类名称。